In [1]:
import os
import argparse
import cv2

import numpy as np
import sys
import glob
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

def TFlite_to_python(model_path, labels, test_path, output_path):
    """
    model_path: enter TFlite model path
    
    labels: enter the list of your model labels
    
    test_path: testing image folder path 
    
    output_path: output of lcd images stored path
    """

    # Path to .tflite file, which contains the model that is used for object detection
    PATH_TO_CKPT = model_path

    # Path to label map f
    
    # Have to do a weird fix for label map if using the COCO "starter model" from
    
    if labels[0] == '???':
        del(labels[0])

    # Load the Tensorflow Lite model.
    # If using Edge TPU, use special load_delegate argument
    interpreter = Interpreter(model_path=PATH_TO_CKPT)

    interpreter.allocate_tensors()

    # Get model details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    floating_model = (input_details[0]['dtype'] == np.float32)

    input_mean = 127.5
    input_std = 127.5

    # Loop over every image and perform detection
    for image_path in os.listdir(test_path):

        # Load image and resize to expected shape [1xHxWx3]
        image = cv2.imread(test_path+"/"+image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        imH, imW, _ = image.shape 
        image_resized = cv2.resize(image_rgb, (width, height))
        input_data = np.expand_dims(image_resized, axis=0)

        # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
        if floating_model:
            input_data = (np.float32(input_data) - input_mean) / input_std

        # Perform the actual detection by running the model with the image as input
        interpreter.set_tensor(input_details[0]['index'],input_data)
        interpreter.invoke()

        # Retrieve detection results
        boxes = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects
        classes = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects
        scores = interpreter.get_tensor(output_details[2]['index'])[0] # Confidence of detected objects
        
        
        score = []
        indexes = []
        for i in np.unique(classes):
            for index in np.where(classes == i):
                print(index)
                for j in index:
                    score.append(scores[j])
                indexes.append(index[score.index(max(score))])
                score.clear()
                
        boxes = np.array([boxes[i] for i in indexes])
        classes = np.array([classes[i] for i in indexes])
        scores = np.array([scores[i] for i in indexes])
        print("classes >>>",classes)
        print("scores >>>", scores)
        print("boxes >>>", boxes)
        
        # Loop over all detections and draw detection box if confidence is above minimum threshold
        for i in range(len(scores)):
            if ((scores[i] > 0.4) and (scores[i] <= 1.0)):

                # Get bounding box coordinates and draw box
                # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
                ymin = int(max(1,(boxes[i][0] * imH)))
                xmin = int(max(1,(boxes[i][1] * imW)))
                ymax = int(min(imH,(boxes[i][2] * imH)))
                xmax = int(min(imW,(boxes[i][3] * imW)))

                cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)
                
                # Draw label
                object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index

                label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
                labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
                label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
                cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
                cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text


                cv2.imwrite(output_path+"/"+image_path, image)

In [2]:
output = r"D:\Projects\Dairy_machine_ocr\Demo\tfliteoutput"  # output path 
model = r"D:/Projects/Dairy_machine_ocr/model-v2 (1).tflite" # TFLITE MODEL PATH
labels = ['lcd','qr'] #labels
test_path = r"D:\Projects\Dairy_machine_ocr\dataset-v1\test" #Test data path 

In [3]:
TFlite_to_python(model, labels, test_path, output) #this will return you the details of image

[0 4]
[1 2 3 5 6 7 8 9]
classes >>> [0. 1.]
scores >>> [0.79971766 0.7580959 ]
boxes >>> [[0.21648431 0.34268412 0.39068717 0.65607154]
 [0.65372723 0.27534887 0.71061844 0.374769  ]]
[1 5]
[0 2 3 4 6 7 8 9]
classes >>> [0. 1.]
scores >>> [0.5727419  0.80007863]
boxes >>> [[0.34934473 0.31040186 0.4209904  0.6236047 ]
 [0.65478927 0.4507725  0.69374555 0.51152974]]
[5 7 8 9]
[0 1 2 3 4 6]
classes >>> [0. 1.]
scores >>> [0.39773974 0.70051396]
boxes >>> [[0.389301   0.36800832 0.45612836 0.5601494 ]
 [0.7163684  0.3811263  0.74916273 0.44229344]]
[0 5]
[1 2 3 4 6 7 8 9]
classes >>> [0. 1.]
scores >>> [0.78572965 0.7258209 ]
boxes >>> [[0.3108514  0.27927002 0.39017946 0.5659238 ]
 [0.7784156  0.6350783  0.81662554 0.70248055]]
[1 2]
[0 3 4 5 6 7 8 9]
classes >>> [0. 1.]
scores >>> [0.76067257 0.8347101 ]
boxes >>> [[0.24699935 0.29711765 0.32081422 0.7129579 ]
 [0.6932564  0.4647185  0.7476865  0.56013215]]
[0 4]
[1 2 3 5 6 7 8 9]
classes >>> [0. 1.]
scores >>> [0.8136501 0.7922268]
box

[0 7]
[1 2 3 4 5 6 8 9]
classes >>> [0. 1.]
scores >>> [0.8738545  0.85360545]
boxes >>> [[0.06510346 0.27357447 0.30621344 0.7869973 ]
 [0.71941674 0.43115914 0.7968445  0.5399726 ]]
[0 3 5]
[1 2 4 6 7 8 9]
classes >>> [0. 1.]
scores >>> [0.81506515 0.75014615]
boxes >>> [[0.01072351 0.21785283 0.24379739 0.7729314 ]
 [0.64384985 0.4252423  0.7076887  0.5444965 ]]
[1 4 9]
[0 2 3 5 6 7 8]
classes >>> [0. 1.]
scores >>> [0.809621 0.828021]
boxes >>> [[0.27103928 0.3045752  0.39165446 0.74986076]
 [0.6837566  0.47073063 0.7341951  0.5426753 ]]
[0]
[1 2 3 4 5 6 7 8 9]
classes >>> [0. 1.]
scores >>> [0.82221353 0.8200656 ]
boxes >>> [[0.22656402 0.34125006 0.36438534 0.68786955]
 [0.7062222  0.44315475 0.7610034  0.5357609 ]]
[1 6]
[0 2 3 4 5 7 8 9]
classes >>> [0. 1.]
scores >>> [0.7988819 0.8107701]
boxes >>> [[0.26559728 0.33558896 0.38543963 0.6332867 ]
 [0.7027205  0.41778648 0.75228    0.49401742]]
[1 6]
[0 2 3 4 5 7 8 9]
classes >>> [0. 1.]
scores >>> [0.7988819 0.8107701]
boxes >>>